In [1]:
using Revise
using LinearAlgebra
using SparseArrays
using Arpack

In [2]:
include("../../src/quantum_utils.jl")
include("../../src/quantum_systems.jl")
using .QuantumUtils
using .QuantumSystems

In [3]:
ϵ = 0.1
â = annihilate(10)
cos(ϵ * (â + â'))

10×10 Matrix{ComplexF64}:
     0.995012+0.0im   2.28402e-16+0.0im  …  -1.10543e-16-0.0im
  2.28402e-16-0.0im      0.985062+0.0im      1.48659e-10+0.0im
   -0.0070358+0.0im   3.17604e-17-0.0im     -4.25499e-17-0.0im
  8.21238e-17-0.0im    -0.0121457-0.0im      -3.39012e-7-0.0im
   2.03106e-5-0.0im   2.01737e-16-0.0im     -8.76711e-17-0.0im
 -1.45668e-16-0.0im    4.53251e-5-0.0im  …   0.000226469+0.0im
  -3.70819e-8+0.0im  -1.90991e-16+0.0im      7.36243e-17+0.0im
  1.45679e-16-0.0im   -9.79694e-8+0.0im       -0.0415844-0.0im
  4.95527e-11-0.0im   1.56535e-16-0.0im      7.89595e-18+0.0im
 -1.10543e-16+0.0im   1.48659e-10-0.0im         0.955633+0.0im

In [4]:
I(10) - (ϵ * (â + â'))^2 / 2 + (ϵ * (â + â'))^4 / 24

10×10 Matrix{ComplexF64}:
    0.995012+0.0im         0.0+0.0im  -0.00703571+0.0im  …          0.0+0.0im
         0.0+0.0im    0.985062+0.0im          0.0+0.0im             0.0+0.0im
 -0.00703571+0.0im         0.0+0.0im     0.975162+0.0im             0.0+0.0im
         0.0+0.0im  -0.0121454+0.0im          0.0+0.0im             0.0+0.0im
  2.04124e-5+0.0im         0.0+0.0im   -0.0171184+0.0im             0.0+0.0im
         0.0+0.0im  4.56435e-5+0.0im          0.0+0.0im  …  0.000229129+0.0im
         0.0+0.0im         0.0+0.0im   7.90569e-5+0.0im             0.0+0.0im
         0.0+0.0im         0.0+0.0im          0.0+0.0im      -0.0415779+0.0im
         0.0+0.0im         0.0+0.0im          0.0+0.0im             0.0+0.0im
         0.0+0.0im         0.0+0.0im          0.0+0.0im        0.955637+0.0im

In [5]:
A = sprand(3, 3, 0.5) 
complex(A)

3×3 SparseMatrixCSC{ComplexF64, Int64} with 5 stored entries:
 0.715295+0.0im           ⋅      0.555671+0.0im
 0.028632+0.0im           ⋅               ⋅    
  0.33672+0.0im  0.383971+0.0im           ⋅    

In [6]:
fill(1:2, 3)

3-element Vector{UnitRange{Int64}}:
 1:2
 1:2
 1:2

In [7]:
X = gate(:X)

2×2 Matrix{Int64}:
 0  1
 1  0

In [8]:
@doc raw"""
    TransmonSystem(;
        ω::Float64=4.4153,  # GHz 
        δ::Float64=0.17215, # GHz
        levels::Int=3,
        frame_ω::Float64=ω,
    ) -> QuantumSystem

Returns a `QuantumSystem` object for a transmon qubit, with the Hamiltonian

```math
H = \omega a^\dagger a + \frac{\delta}{2} a^\dagger a^\dagger a a 
```

where `a` is the annihilation operator.

# Keyword Arguments
- `ω`: The frequency of the transmon, in GHz.
- `δ`: The anharmonicity of the transmon, in GHz.
- `levels`: The number of levels in the transmon.
- `frame_ω`: The frequency of the rotating frame, in GHz.

"""
function TransmonSystem(;
    ω::Float64=4.0,  # GHz 
    δ::Float64=-0.2, # GHz
    levels::Int=3,
    lab_frame::Bool=false,
    frame_ω::Float64=lab_frame ? 0.0 : ω,
) 
    if lab_frame
        frame_ω = 0.0
    end
    
    if !(frame_ω ≈ 0.0)
        lab_frame = false
    end

    a = annihilate(levels)

    if lab_frame 
        # TODO: add functionality for hamiltonian with cosine
        H_drift = ω * a' * a + δ / 12 * (a + a')^4 
    else
        H_drift = (ω - frame_ω) * a' * a + δ / 2 * a' * a' * a * a
    end

    H_drives = [a + a', 1.0im * (a - a')]

    params = Dict{Symbol, Any}(
        :ω => ω,
        :δ => δ,
        :levels => levels,
        :lab_frame => lab_frame,
        :frame_ω => frame_ω,
    )

    return QuantumSystem(
        H_drift,
        H_drives;
        constructor=TransmonSystem,
        params=params,
    )
end

TransmonSystem

In [9]:
sys = TransmonSystem()
sys.params

Dict{Symbol, Any} with 5 entries:
  :ω         => 4.0
  :lab_frame => false
  :δ         => -0.2
  :levels    => 3
  :frame_ω   => 4.0

In [10]:
sys = TransmonSystem()
new_sys = sys(lab_frame=true)
new_sys.params

Dict{Symbol, Any} with 5 entries:
  :ω         => 4.0
  :lab_frame => true
  :δ         => -0.2
  :levels    => 3
  :frame_ω   => 0.0

In [11]:
sys.H_drift

3×3 SparseMatrixCSC{ComplexF64, Int64} with 1 stored entry:
     ⋅          ⋅           ⋅    
     ⋅          ⋅           ⋅    
     ⋅          ⋅      -0.2+0.0im

In [12]:
new_sys.H_drift

3×3 SparseMatrixCSC{ComplexF64, Int64} with 5 stored entries:
      -0.05+0.0im       ⋅      -0.0707107+0.0im
            ⋅      3.85+0.0im             ⋅    
 -0.0707107+0.0im       ⋅             7.9+0.0im

In [13]:
function TransmonDipoleCoupling(
    g_ij::Float64,
    pair::Tuple{Int, Int},
    sub_levels::Vector{Int};
    lab_frame::Bool=false,
)
    i, j = pair
    a_i = lift(annihilate(sub_levels[i]), i, sub_levels)
    a_j = lift(annihilate(sub_levels[j]), j, sub_levels)

    if lab_frame
        op = g_ij * (a_i + a_i') * (a_j + a_j')
    else
        op = g_ij * (a_i * a_j' + a_i' * a_j)
    end

    params = Dict{Symbol, Any}(
        :lab_frame => lab_frame,
    )

    return QuantumSystemCoupling(
        op,
        g_ij,
        pair,
        sub_levels,
        TransmonDipoleCoupling,
        params
    )
end

function TransmonDipoleCoupling(
    g_ij::Float64,
    pair::Tuple{Int, Int},
    sub_systems::Vector{QuantumSystem};
    kwargs...
)
    sub_levels = [sys.levels for sys ∈ sub_systems]
    return TransmonDipoleCoupling(g_ij, pair, sub_levels; kwargs...)
end


TransmonDipoleCoupling (generic function with 2 methods)

In [14]:
coupling = TransmonDipoleCoupling(1.0, (1, 2), [3, 4])

QuantumSystemCoupling(sparse([5, 6, 7, 2, 3, 9, 4, 10, 11, 6, 7, 8], [2, 3, 4, 5, 6, 6, 7, 7, 8, 9, 10, 11], ComplexF64[1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.7320508075688772 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im, 1.7320508075688772 + 0.0im, 2.0000000000000004 + 0.0im, 2.4494897427831783 + 0.0im, 1.4142135623730951 + 0.0im, 2.0000000000000004 + 0.0im, 2.4494897427831783 + 0.0im], 12, 12), 1.0, (1, 2), [3, 4], TransmonDipoleCoupling, Dict{Symbol, Any}(:lab_frame => false))

In [15]:
sys1 = copy(sys)

QuantumSystem(sparse([3], [3], ComplexF64[-0.20000000000000004 + 0.0im], 3, 3), SparseMatrixCSC{ComplexF64, Int64}[sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im], 3, 3), sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[-0.0 - 1.0im, 0.0 + 1.0im, -0.0 - 1.4142135623730951im, 0.0 + 1.4142135623730951im], 3, 3)], sparse([6, 3], [3, 6], [0.20000000000000004, -0.20000000000000004], 6, 6), SparseMatrixCSC{Float64, Int64}[sparse([5, 4, 6, 5, 2, 1, 3, 2], [1, 2, 2, 3, 4, 5, 5, 6], [-1.0, -1.0, -1.4142135623730951, -1.4142135623730951, 1.0, 1.0, 1.4142135623730951, 1.4142135623730951], 6, 6), sparse([2, 1, 3, 2, 5, 4, 6, 5], [1, 2, 2, 3, 4, 5, 5, 6], [-1.0, 1.0, -1.4142135623730951, 1.4142135623730951, -1.0, 1.0, -1.4142135623730951, 1.4142135623730951], 6, 6)], 3, TransmonSystem, Dict{Symbol, Any}(:ω => 4.0, :lab_frame => false, :δ => -0.2, :levels => 3, :frame_ω => 4.0))

In [16]:
sys2 = copy(sys)

QuantumSystem(sparse([3], [3], ComplexF64[-0.20000000000000004 + 0.0im], 3, 3), SparseMatrixCSC{ComplexF64, Int64}[sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im], 3, 3), sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[-0.0 - 1.0im, 0.0 + 1.0im, -0.0 - 1.4142135623730951im, 0.0 + 1.4142135623730951im], 3, 3)], sparse([6, 3], [3, 6], [0.20000000000000004, -0.20000000000000004], 6, 6), SparseMatrixCSC{Float64, Int64}[sparse([5, 4, 6, 5, 2, 1, 3, 2], [1, 2, 2, 3, 4, 5, 5, 6], [-1.0, -1.0, -1.4142135623730951, -1.4142135623730951, 1.0, 1.0, 1.4142135623730951, 1.4142135623730951], 6, 6), sparse([2, 1, 3, 2, 5, 4, 6, 5], [1, 2, 2, 3, 4, 5, 5, 6], [-1.0, 1.0, -1.4142135623730951, 1.4142135623730951, -1.0, 1.0, -1.4142135623730951, 1.4142135623730951], 6, 6)], 3, TransmonSystem, Dict{Symbol, Any}(:ω => 4.0, :lab_frame => false, :δ => -0.2, :levels => 3, :frame_ω => 4.0))

In [17]:
coupling = TransmonDipoleCoupling(1.0, (1, 2), [sys1, sys2])

QuantumSystemCoupling(sparse([4, 5, 2, 3, 7, 8, 5, 6], [2, 3, 4, 5, 5, 6, 7, 8], ComplexF64[1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im, 2.0000000000000004 + 0.0im, 1.4142135623730951 + 0.0im, 2.0000000000000004 + 0.0im], 9, 9), 1.0, (1, 2), [3, 3], TransmonDipoleCoupling, Dict{Symbol, Any}(:lab_frame => false))

In [18]:
coupling.op

9×9 SparseMatrixCSC{ComplexF64, Int64} with 8 stored entries:
     ⋅          ⋅              ⋅      …          ⋅          ⋅          ⋅    
     ⋅          ⋅              ⋅                 ⋅          ⋅          ⋅    
     ⋅          ⋅              ⋅                 ⋅          ⋅          ⋅    
     ⋅      1.0+0.0im          ⋅                 ⋅          ⋅          ⋅    
     ⋅          ⋅      1.41421+0.0im     1.41421+0.0im      ⋅          ⋅    
     ⋅          ⋅              ⋅      …          ⋅      2.0+0.0im      ⋅    
     ⋅          ⋅              ⋅                 ⋅          ⋅          ⋅    
     ⋅          ⋅              ⋅                 ⋅          ⋅          ⋅    
     ⋅          ⋅              ⋅                 ⋅          ⋅          ⋅    

In [19]:
composite_sys = CompositeQuantumSystem([sys1, sys2]; couplings=[coupling])

CompositeQuantumSystem(sparse([4, 3, 5, 2, 3, 7, 6, 8, 5, 7, 6, 8, 9], [2, 3, 3, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9], ComplexF64[1.0 + 0.0im, -0.20000000000000004 + 0.0im, 1.4142135623730951 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im, -0.20000000000000004 + 0.0im, 2.0000000000000004 + 0.0im, 1.4142135623730951 + 0.0im, -0.20000000000000004 + 0.0im, 2.0000000000000004 + 0.0im, -0.20000000000000004 + 0.0im, -0.4000000000000001 + 0.0im], 9, 9), SparseMatrixCSC{ComplexF64, Int64}[sparse([4, 5, 6, 1, 7, 2, 8, 3, 9, 4, 5, 6], [1, 2, 3, 4, 4, 5, 5, 6, 6, 7, 8, 9], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im], 9, 9), sparse([4, 5, 6, 1, 7, 2, 8, 3, 9, 4, 5, 6], [1, 2, 3, 4, 4, 5, 5, 6, 6, 7, 8, 9], ComplexF64[0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 + 1.0i

In [20]:
composite_sys.H_drift

9×9 SparseMatrixCSC{ComplexF64, Int64} with 13 stored entries:
     ⋅          ⋅              ⋅          ⋅      …       ⋅           ⋅    
     ⋅          ⋅              ⋅      1.0+0.0im          ⋅           ⋅    
     ⋅          ⋅         -0.2+0.0im      ⋅              ⋅           ⋅    
     ⋅      1.0+0.0im          ⋅          ⋅              ⋅           ⋅    
     ⋅          ⋅      1.41421+0.0im      ⋅              ⋅           ⋅    
     ⋅          ⋅              ⋅          ⋅      …   2.0+0.0im       ⋅    
     ⋅          ⋅              ⋅          ⋅              ⋅           ⋅    
     ⋅          ⋅              ⋅          ⋅         -0.2+0.0im       ⋅    
     ⋅          ⋅              ⋅          ⋅              ⋅      -0.4+0.0im

In [21]:
composite_sys.H_drives[3]

9×9 SparseMatrixCSC{ComplexF64, Int64} with 12 stored entries:
     ⋅          1.0+0.0im          ⋅      …          ⋅              ⋅    
 1.0+0.0im          ⋅      1.41421+0.0im             ⋅              ⋅    
     ⋅      1.41421+0.0im          ⋅                 ⋅              ⋅    
     ⋅              ⋅              ⋅                 ⋅              ⋅    
     ⋅              ⋅              ⋅                 ⋅              ⋅    
     ⋅              ⋅              ⋅      …          ⋅              ⋅    
     ⋅              ⋅              ⋅             1.0+0.0im          ⋅    
     ⋅              ⋅              ⋅                 ⋅      1.41421+0.0im
     ⋅              ⋅              ⋅         1.41421+0.0im          ⋅    

In [22]:
composite_sys.subsystems

2-element Vector{QuantumSystem}:
 QuantumSystem(sparse([3], [3], ComplexF64[-0.20000000000000004 + 0.0im], 3, 3), SparseMatrixCSC{ComplexF64, Int64}[sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im], 3, 3), sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[-0.0 - 1.0im, 0.0 + 1.0im, -0.0 - 1.4142135623730951im, 0.0 + 1.4142135623730951im], 3, 3)], sparse([6, 3], [3, 6], [0.20000000000000004, -0.20000000000000004], 6, 6), SparseMatrixCSC{Float64, Int64}[sparse([5, 4, 6, 5, 2, 1, 3, 2], [1, 2, 2, 3, 4, 5, 5, 6], [-1.0, -1.0, -1.4142135623730951, -1.4142135623730951, 1.0, 1.0, 1.4142135623730951, 1.4142135623730951], 6, 6), sparse([2, 1, 3, 2, 5, 4, 6, 5], [1, 2, 2, 3, 4, 5, 5, 6], [-1.0, 1.0, -1.4142135623730951, 1.4142135623730951, -1.0, 1.0, -1.4142135623730951, 1.4142135623730951], 6, 6)], 3, TransmonSystem, Dict{Symbol, Any}(:ω => 4.0, :lab_frame => false, :δ => -0.2, :levels => 3, :frame_ω => 4.0))
 QuantumSys

In [23]:
coupling = TransmonDipoleCoupling(1.0, (1, 2), [sys1, sys2])

QuantumSystemCoupling(sparse([4, 5, 2, 3, 7, 8, 5, 6], [2, 3, 4, 5, 5, 6, 7, 8], ComplexF64[1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.4142135623730951 + 0.0im, 2.0000000000000004 + 0.0im, 1.4142135623730951 + 0.0im, 2.0000000000000004 + 0.0im], 9, 9), 1.0, (1, 2), [3, 3], TransmonDipoleCoupling, Dict{Symbol, Any}(:lab_frame => false))

In [24]:
composite_sys = CompositeQuantumSystem([sys1, sys2]; couplings=[coupling])
for sys in composite_sys.subsystems
    println(sys.params)
end

Dict{Symbol, Any}(:ω => 4.0, :lab_frame => false, :δ => -0.2, :levels => 3, :frame_ω => 4.0)
Dict{Symbol, Any}(:ω => 4.0, :lab_frame => false, :δ => -0.2, :levels => 3, :frame_ω => 4.0)


In [25]:
new_composite_sys = composite_sys(
    subsystem_params=Dict(
        1 => Dict(
            :δ => -0.3,
            :levels => 4,
        ),
    ),
    coupling_params=Dict(
        1 => Dict(
            :g_ij => 0.2
        ),
    )
)

CompositeQuantumSystem(sparse([4, 3, 5, 2, 3, 7, 6, 8, 5, 7, 6, 8, 10, 9, 11, 8, 10, 9, 11, 12], [2, 3, 3, 4, 5, 5, 6, 6, 7, 7, 8, 8, 8, 9, 9, 10, 10, 11, 11, 12], ComplexF64[0.2 + 0.0im, -0.20000000000000004 + 0.0im, 0.28284271247461906 + 0.0im, 0.2 + 0.0im, 0.28284271247461906 + 0.0im, 0.28284271247461906 + 0.0im, -0.20000000000000004 + 0.0im, 0.40000000000000013 + 0.0im, 0.28284271247461906 + 0.0im, -0.30000000000000004 + 0.0im, 0.40000000000000013 + 0.0im, -0.30000000000000004 + 0.0im, 0.34641016151377546 + 0.0im, -0.5000000000000001 + 0.0im, 0.4898979485566357 + 0.0im, 0.34641016151377546 + 0.0im, -0.9 + 0.0im, 0.4898979485566357 + 0.0im, -0.9 + 0.0im, -1.1 + 0.0im], 12, 12), SparseMatrixCSC{ComplexF64, Int64}[sparse([4, 5, 6, 1, 7, 2, 8, 3, 9, 4, 10, 5, 11, 6, 12, 7, 8, 9], [1, 2, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 11, 12], ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.0 + 0.0im, 1.414

In [26]:
for sys in new_composite_sys.subsystems
    println(sys.params)
end

Dict{Symbol, Any}(:ω => 4.0, :lab_frame => false, :δ => -0.3, :levels => 4, :frame_ω => 4.0)
Dict{Symbol, Any}(:ω => 4.0, :lab_frame => false, :δ => -0.2, :levels => 3, :frame_ω => 4.0)


In [27]:
embed(X, sys1)

3×3 Matrix{ComplexF64}:
 0.0+0.0im  1.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im

In [35]:
X_embeded = embed(X, composite_sys, 2)
sparse(X_embeded)

9×9 SparseMatrixCSC{ComplexF64, Int64} with 4 stored entries:
     ⋅          ⋅          ⋅      …      ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅             ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅             ⋅          ⋅          ⋅    
 1.0+0.0im      ⋅          ⋅             ⋅          ⋅          ⋅    
     ⋅      1.0+0.0im      ⋅             ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅      …      ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅             ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅             ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅             ⋅          ⋅          ⋅    

In [34]:
ssids = get_subspace_indices([1:2, 1:2], composite_sys.sub_levels) 
X_embeded[ssids, ssids]

4×4 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im  1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  1.0+0.0im
 1.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im  0.0+0.0im  0.0+0.0im